<a href="https://colab.research.google.com/github/jenmusdat/Project3_BeerQuality/blob/main/Analysis/training_taste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:13 https://developer.download.n

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Start a SparkSession
from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("word2vec").getOrCreate()

In [4]:
MAX_MEMORY = "5g"

spark = SparkSession \
    .builder \
    .appName("word2vec") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .config("spark.driver.maxResultSize", MAX_MEMORY)\
    .getOrCreate()

In [5]:
# Import needed libs
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.ml.feature import Word2Vec, Word2VecModel
import string


In [6]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="/content/gdrive/MyDrive/Project3_BeerQuality/Resources/reviews_beer_brewery.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("reviews_beer_brewery.csv"), header=True)


In [7]:
# remove alpha characters from data set on review numerics
from pyspark.sql.functions import regexp_extract, col
pattern = r'^\d+.\d+$'
df = df.filter(regexp_extract(col('look'), pattern, 0) != '')

In [8]:
df = df.filter(regexp_extract(col('smell'), pattern, 0) != '')
df = df.filter(regexp_extract(col('taste'), pattern, 0) != '')
df = df.filter(regexp_extract(col('overall'), pattern, 0) != '')
df = df.filter(regexp_extract(col('score'), pattern, 0) != '')
df = df.filter(regexp_extract(col('feel'), pattern, 0) != '')

In [9]:
# Next 6 lines casts to floats all the review ratings
df = df.withColumn('look_numeric', df['look'].cast('float'))

In [10]:
df = df.withColumn('smell_numeric', df['smell'].cast('float'))

In [11]:
df = df.withColumn('taste_numeric', df['taste'].cast('float'))

In [12]:
df = df.withColumn('feel_numeric', df['feel'].cast('float'))

In [13]:
df = df.withColumn('overall_numeric', df['overall'].cast('float'))

In [14]:
df = df.withColumn('score_numeric', df['score'].cast('float'))

In [15]:
# Confirm data types
df.printSchema

<bound method DataFrame.printSchema of DataFrame[beer_id: string, username: string, date: string, text: string, look: string, smell: string, taste: string, feel: string, overall: string, score: string, beer_name: string, review_state: string, style: string, availability: string, abv: string, brewery_id: string, brewery_name: string, brewery_city: string, brewery_state: string, brewery_types: string, look_numeric: float, smell_numeric: float, taste_numeric: float, feel_numeric: float, overall_numeric: float, score_numeric: float]>

In [16]:
# functions for manipulating review text into a format we can tokenize
def remove_punctuation(txt):
  txt = txt.strip('\xa0\xa0 ')
  return "".join(l if l not in string.punctuation else "" for l in txt)

def remove_space (text):
  if text:
    return text.replace(' ', '')
  else:
    return 'Unknown'
  

In [17]:
# define the udfs for the data
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, udf

remove_punctuation_udf = udf(remove_punctuation, StringType())
remove_punctuation_udf

remove_space_udf = udf(remove_space, StringType())
remove_space_udf

<function __main__.remove_space>

In [18]:
# clean the beer name by removing spaces (will be combined with text)
beer_df = df.withColumn('beer_clean', remove_space_udf(col('beer_name')))
# beer_df.show()

In [19]:
# clean the style name by removing spaces (will be combined with text)
style_df = beer_df.withColumn('style_clean', remove_space_udf(col('style')))
# style_df.show()

In [20]:
# clean the text by removing punctuation
text_df = style_df.withColumn("clean_text", remove_punctuation_udf(col("text")))
text_df.show()

+-------+----------------+----------+--------------------+----+-----+-----+----+-------+-----+--------------------+------------+--------------------+--------------------+----+----------+--------------------+------------+-------------+-------------+------------+-------------+-------------+------------+---------------+-------------+--------------------+--------------------+--------------------+
|beer_id|        username|      date|                text|look|smell|taste|feel|overall|score|           beer_name|review_state|               style|        availability| abv|brewery_id|        brewery_name|brewery_city|brewery_state|brewery_types|look_numeric|smell_numeric|taste_numeric|feel_numeric|overall_numeric|score_numeric|          beer_clean|         style_clean|          clean_text|
+-------+----------------+----------+--------------------+----+-----+-----+----+-------+-----+--------------------+------------+--------------------+--------------------+----+----------+--------------------+-

In [21]:
# Combine beer_name, style, and review text into a Final_text column
from pyspark.sql.functions import concat_ws

combined_df = text_df.withColumn('Final_text', concat_ws(' ',text_df.beer_clean, text_df.style_clean, text_df.clean_text))
combined_df.select('text', 'Final_text').show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [22]:
# Setup and execute Tokenizer and Stop words pipeline
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml import Pipeline

# "Creating pipeline..."
tokenizer = Tokenizer(inputCol="Final_text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text', outputCol='features')

pipeline = Pipeline(stages=[tokenizer, stopremove])

# "Training model..."
pipeline_stg = pipeline.fit(combined_df)
final_df = pipeline_stg.transform(combined_df)


In [ ]:
# final_df.show()

In [23]:
final_df = final_df.select('taste_numeric','features')

In [25]:
final_df.show()

+-------------+--------------------+
|taste_numeric|            features|
+-------------+--------------------+
|          4.0|[motorbreathimper...|
|          3.5|[paybackpilsner, ...|
|         4.25|[paybackpilsner, ...|
|          3.5|[paybackpilsner, ...|
|          3.5|[paybackpilsner, ...|
|          3.5|[paybackpilsner, ...|
|         3.25|[paybackpilsner, ...|
|          4.0|[paybackpilsner, ...|
|          3.5|[paybackpilsner, ...|
|          3.5|[paybackpilsner, ...|
|          3.5|[hellhammer, ryeb...|
|          4.5|[hellhammer, ryeb...|
|          3.0|[vanillaporter, a...|
|         3.75|[vanillaporter, a...|
|         3.75|[vanillaporter, a...|
|          3.5|[vanillaporter, a...|
|          2.5|[vanillaporter, a...|
|          4.0|[vanillaporter, a...|
|          4.0|[vanillaporter, a...|
|          3.5|[vanillaporter, a...|
+-------------+--------------------+
only showing top 20 rows



In [26]:
# Read the model from disk to confirm
w2v_loaded_model = Word2VecModel.load('/content/gdrive/MyDrive/Project3_BeerQuality/review.md')

In [27]:
# check to see if synonym is consistent with loaded model
w2v_loaded_model.findSynonymsArray("light", 5)

[('mild', 0.6426884531974792),
 ('faint', 0.6223998665809631),
 ('slight', 0.6191715002059937),
 ('mostly', 0.6169756650924683),
 ('medium', 0.6095581650733948)]

In [28]:
# Transform the mode
w2v_model_df = w2v_loaded_model.transform(final_df)

In [29]:
w2v_model_df.show()

+-------------+--------------------+--------------------+
|taste_numeric|            features|               model|
+-------------+--------------------+--------------------+
|          4.0|[motorbreathimper...|[0.02950784611367...|
|          3.5|[paybackpilsner, ...|[-0.0872463285923...|
|         4.25|[paybackpilsner, ...|[0.01790586973612...|
|          3.5|[paybackpilsner, ...|[-0.1276244542612...|
|          3.5|[paybackpilsner, ...|[-0.0131208907440...|
|          3.5|[paybackpilsner, ...|[-0.0353247860187...|
|         3.25|[paybackpilsner, ...|[-0.0606238029503...|
|          4.0|[paybackpilsner, ...|[0.00897214795847...|
|          3.5|[paybackpilsner, ...|[0.02342551416502...|
|          3.5|[paybackpilsner, ...|[-0.0209101041158...|
|          3.5|[hellhammer, ryeb...|[0.01432307037573...|
|          4.5|[hellhammer, ryeb...|[-0.1081325657665...|
|          3.0|[vanillaporter, a...|[-0.1101182210924...|
|         3.75|[vanillaporter, a...|[-0.0197092709394...|
|         3.75

In [30]:
w2v_model_df = w2v_model_df.select('taste_numeric', 'model')

In [31]:
# Prepare the hyperparameters
from pyspark.ml.regression import LinearRegression, LinearRegressionModel
lin_model = LinearRegression(
   maxIter=5,
   regParam=0.3,
   featuresCol="model",
   labelCol="taste_numeric"
)

In [32]:
w2v_model_df.cache()
w2v_model_df.show()

+-------------+--------------------+
|taste_numeric|               model|
+-------------+--------------------+
|          4.0|[0.02950784611367...|
|          3.5|[-0.0872463285923...|
|         4.25|[0.01790586973612...|
|          3.5|[-0.1276244542612...|
|          3.5|[-0.0131208907440...|
|          3.5|[-0.0353247860187...|
|         3.25|[-0.0606238029503...|
|          4.0|[0.00897214795847...|
|          3.5|[0.02342551416502...|
|          3.5|[-0.0209101041158...|
|          3.5|[0.01432307037573...|
|          4.5|[-0.1081325657665...|
|          3.0|[-0.1101182210924...|
|         3.75|[-0.0197092709394...|
|         3.75|[-0.0049533498369...|
|          3.5|[0.02202844081446...|
|          2.5|[-0.0874559852428...|
|          4.0|[-0.0728758174786...|
|          4.0|[-0.1125914826324...|
|          3.5|[-0.1178625054068...|
+-------------+--------------------+
only showing top 20 rows



In [33]:
# split training and testing for the model (for temporary make the training set small as processing takes time)
training, testing = w2v_model_df.randomSplit([0.80, 0.20])

In [34]:
training.show()

+-------------+--------------------+
|taste_numeric|               model|
+-------------+--------------------+
|          1.0|[-0.3051145498951...|
|          1.0|[-0.2089737124741...|
|          1.0|[-0.1599104826649...|
|          1.0|[-0.1488741986453...|
|          1.0|[-0.1452391079316...|
|          1.0|[-0.1416912284815...|
|          1.0|[-0.1338947367233...|
|          1.0|[-0.1293812746912...|
|          1.0|[-0.1291604785341...|
|          1.0|[-0.1286499396006...|
|          1.0|[-0.1245115498701...|
|          1.0|[-0.1097804032266...|
|          1.0|[-0.1021585103888...|
|          1.0|[-0.1004139792794...|
|          1.0|[-0.0961883496493...|
|          1.0|[-0.0957816023534...|
|          1.0|[-0.0949192517682...|
|          1.0|[-0.0943613576819...|
|          1.0|[-0.0924596638641...|
|          1.0|[-0.0893084063427...|
+-------------+--------------------+
only showing top 20 rows



In [35]:
training_pandas_df = training.toPandas()

In [36]:
training_pandas_df.isnull().sum()

taste_numeric    0
model            0
dtype: int64

In [37]:
training_pandas_df = training_pandas_df.dropna()

In [38]:
training_pandas_df.head()

,taste_numeric,model
0,1.0,"[-0.3051145498951276, -0.010960971315701801, 0..."
1,1.0,"[-0.20897371247410776, -0.007770388945937157, ..."
2,1.0,"[-0.159910482664903, -0.16529758125155541, -0...."
3,1.0,"[-0.1488741986453533, -0.05295970737934113, -0..."
4,1.0,"[-0.14523910793165368, 0.12042828451505759, -0..."


In [39]:
training_pandas_df.dtypes

taste_numeric    float32
model             object
dtype: object

In [40]:
from sklearn.linear_model import LinearRegression
pandas_lin_model = LinearRegression()

In [41]:
lin_taste_train_model = pandas_lin_model.fit(training_pandas_df['model'].to_list(),training_pandas_df['taste_numeric'])

In [42]:
lin_taste_train_model.score(training_pandas_df['model'].to_list(), training_pandas_df['taste_numeric'])

0.40117698054002715

In [43]:
lin_taste_train_model.coef_

array([ 0.36290024,  1.10114961,  1.04387868, -0.35164705,  0.43332021,
        1.58799075,  1.19258722,  1.37185673, -0.02210624, -0.2958749 ,
        0.19552785, -0.38463898,  1.04797673,  0.49026115,  0.56493662,
        0.59401674, -0.80571681,  0.55760469, -0.10462712, -0.78373299,
       -0.28882318, -0.79092097,  0.02946115, -0.38715856,  0.42091326,
       -0.2294125 , -0.29099017, -0.77410423, -0.7831629 ,  1.11745276,
        0.83511485, -0.33832907,  0.58042255, -0.08703434,  0.37457563,
       -0.96475182, -0.23768364,  0.40861255,  0.24617174, -0.08592977,
        0.09683945, -1.05099349,  0.7860532 , -0.79394493, -0.07657311,
       -0.32449919, -0.79990001, -1.09671626, -1.81149177, -0.26168971,
        0.05748853,  1.1350091 ,  0.5598007 ,  0.3560846 , -1.08390276,
        0.48646536, -0.32942315, -0.05073907, -0.73582004,  0.68335764,
        0.4881604 ,  0.26190334, -0.60572494,  1.41062626,  0.88030838,
       -0.61668714, -0.26849134, -0.73671721, -0.17684479, -0.01

In [44]:
# Fit the taste review to a model (takes time)
lin_taste_train_model = lin_model.fit(training)

In [45]:
# Save the model for later use
lin_taste_train_model.write().overwrite().save("/content/gdrive/MyDrive/Project3_BeerQuality/lin_taste_train.md")

In [47]:
# Read the model
loaded_lin_taste_train_model = LinearRegressionModel.load("/content/gdrive/MyDrive/Project3_BeerQuality/lin_taste_train.md")

In [48]:
# Transform the training data to peek at the predictions
lin_taste_df = loaded_lin_taste_train_model.transform(training)

In [50]:
lin_taste_df.show()

+-------------+--------------------+------------------+
|taste_numeric|               model|        prediction|
+-------------+--------------------+------------------+
|          1.0|[-0.3051145498951...|2.9548818184403576|
|          1.0|[-0.2089737124741...|3.5625522611407567|
|          1.0|[-0.1599104826649...| 3.657356522035352|
|          1.0|[-0.1488741986453...|3.4408712437529894|
|          1.0|[-0.1452391079316...| 3.297180027869974|
|          1.0|[-0.1416912284815...| 5.408717355281279|
|          1.0|[-0.1338947367233...|2.9306978837061983|
|          1.0|[-0.1293812746912...|2.9173156978740042|
|          1.0|[-0.1291604785341...|3.0887632712369246|
|          1.0|[-0.1286499396006...|3.4429453474960585|
|          1.0|[-0.1245115498701...|1.2737030696298972|
|          1.0|[-0.1097804032266...|2.8596054639490216|
|          1.0|[-0.1021585103888...|2.8526111393897904|
|          1.0|[-0.1004139792794...|2.6804751029559406|
|          1.0|[-0.0961883496493...|3.2140264322

In [51]:
# show a sample of the predictions
lin_taste_df.select('taste_numeric', 'prediction').show(truncate=False)

+-------------+------------------+
|taste_numeric|prediction        |
+-------------+------------------+
|1.0          |2.9548818184403576|
|1.0          |3.5625522611407567|
|1.0          |3.657356522035352 |
|1.0          |3.4408712437529894|
|1.0          |3.297180027869974 |
|1.0          |5.408717355281279 |
|1.0          |2.9306978837061983|
|1.0          |2.9173156978740042|
|1.0          |3.0887632712369246|
|1.0          |3.4429453474960585|
|1.0          |1.2737030696298972|
|1.0          |2.8596054639490216|
|1.0          |2.8526111393897904|
|1.0          |2.6804751029559406|
|1.0          |3.2140264322821057|
|1.0          |3.5488809558723355|
|1.0          |3.2528086509341922|
|1.0          |3.1322177212021627|
|1.0          |3.359119238380456 |
|1.0          |3.462644167031043 |
+-------------+------------------+
only showing top 20 rows



In [55]:
gr = lin_taste_df.groupBy("taste_numeric").count()
gr.show()

+-------------+------+
|taste_numeric| count|
+-------------+------+
|          5.0| 76025|
|          2.5| 39044|
|          2.0| 24118|
|          3.0|108179|
|         2.75|  9750|
|          1.5|  8605|
|         3.75|102154|
|         1.75|  1556|
|         4.25|129405|
|          3.5|249266|
|          1.0|  8010|
|         2.25|  3064|
|         4.75| 44853|
|          4.5|289785|
|         1.25|   906|
|         3.25| 27961|
|          4.0|456479|
+-------------+------+



In [56]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!
